In [1]:
proj_list =[
    'boringssl', 'c-ares',
    'freetype2', 'guetzli',
    'harfbuzz', 'libpng',
    'libssh', 'libxml2',
    'pcre', 'proj4',
    'r32', 'sqlite3',
    'vorbis', 'woff2',
    'wpantund'
]

In [2]:
target_project = 0

In [3]:
from sklearn.model_selection import train_test_split
import torch
import torch, gc

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data
import data_loader as dl
import initializer as init
import trainer
import tester
# import predictor
import model_util as mu

import os

In [4]:
gc.collect()
torch.cuda.empty_cache()

# print(torch.cuda.memory_summary(device=None, abbreviated=False))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
'''
prefix = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 134, 56, 2, 47]
postfix = [2, 124, 88, 48, 48, 2, 47, 89, 48, 2, 47, 47, 88, 48, 48, 2, 47, 89, 48, 2, 47, 47, 49, 48, 2, 56, 134, 91, 2, 121, 91, 2, 121, 91, 2, 2, 47, 2, 56, 134, 128, 50, 88, 48, 2, 47, 2, 124, 88, 48, 48, 2, 47, 89, 48, 2, 47, 47, 49, 48, 2, 121, 91, 2]
label-type = [2]
label-prefix = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
label-postfix = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
case = 2
'''

'\nprefix = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 134, 56, 2, 47]\npostfix = [2, 124, 88, 48, 48, 2, 47, 89, 48, 2, 47, 47, 88, 48, 48, 2, 47, 89, 48, 2, 47, 47, 49, 48, 2, 56, 134, 91, 2, 121, 91, 2, 121, 91, 2, 2, 47, 2, 56, 134, 128, 50, 88, 48, 2, 47, 2, 124, 88, 48, 48, 2, 47, 89, 48, 2, 47, 47, 49, 48, 2, 121, 91, 2]\nlabel-type = [2]\nlabel-prefix = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]\nlabel-postfix = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]\ncase = 2\n'

In [6]:
# get all data exept target project
prefix_np, postfix_np,\
label_np, label_prefix_np,\
label_postfix_np, case_np = data.getTrainData(proj_list, proj_list[target_project])

Getting data for "boringssl" from "boringssl"
Getting data for "boringssl" from "c-ares"
Getting data for "boringssl" from "freetype2"
Getting data for "boringssl" from "guetzli"
Getting data for "boringssl" from "harfbuzz"
Getting data for "boringssl" from "libpng"
Getting data for "boringssl" from "libssh"
Getting data for "boringssl" from "libxml2"
Getting data for "boringssl" from "pcre"
Getting data for "boringssl" from "proj4"
Getting data for "boringssl" from "r32"
Getting data for "boringssl" from "sqlite3"
Getting data for "boringssl" from "vorbis"
Getting data for "boringssl" from "woff2"
Getting data for "boringssl" from "wpantund"


In [7]:
# get target project data
test_prefix, test_postfix,\
test_label, test_label_prefix,\
test_label_postfix, test_case = data.getTestData(proj_list[target_project])

In [8]:
train_prefix, val_prefix,\
train_postfix, val_postfix,\
train_label, val_label,\
train_label_prefix, val_label_prefix,\
train_label_postfix, val_label_postfix,\
train_case, val_case = train_test_split(
    prefix_np, postfix_np,\
    label_np, label_prefix_np,\
    label_postfix_np, case_np,\
    test_size = 0.2, random_state = 43
)

In [9]:
train_dataloader, val_dataloader, test_dataloader =\
    dl.data_loader(
        train_prefix, val_prefix,
        train_postfix, val_postfix,
        train_label, val_label,
        train_label_prefix, val_label_prefix,
        train_label_postfix, val_label_postfix,
        train_case, val_case,


        test_prefix, test_postfix,
        test_label, test_label_prefix,
        test_label_postfix, test_case,

        batch_size=1000
    )

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [11]:
# ====================
# set parameters here
# ====================

overall_title = 'fc3'
title = overall_title + '_' + proj_list[target_project]
epochs = 20 

# max_len, source_code_tokens, token_choices = data.getInfo()

learning_rate = 0.001
weight_decay = 0.0

embed_dim = 100 # 100
hidden_size = 200 # 200
n_layers = 1
output_size = 1 # max(token_choices) + 1
dropout = 0.0
max_length = 64 # max_len
input_size = 214 # max(token_choices) + 1
device = device

model_name = "seq2seq"
optim_name = "Adam"
loss_fn_name = "BCE"

teacher_forcing_ratio = 0.75
threshold = torch.tensor([0.5]).to(device)

In [12]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/'+title+'/tests')

In [13]:
trainer.set_seed(42)

model, loss_fn, optimizer = init.initialize_model(
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    n_layers=n_layers,
    output_size=output_size,
    dropout=dropout,
    max_length=max_length,
    input_size=input_size,
    device=device,
    loss_fn_name=loss_fn_name
)

print(model)

MySeq2Seq(
  (prefixEncoder): Encoder(
    (embedding): Embedding(214, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (postfixEncoder): Encoder(
    (embedding): Embedding(214, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(214, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
  )
  (pointer): PointerNetwork(
    (fc1): Linear(in_features=800, out_features=400, bias=True)
    (fc2): Linear(in_features=400, out_features=200, bias=True)
    (fc3): Linear(in_features=200, out_features=1, bias=True)
    (dp1): Dropou

In [14]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    teacher_forcing_ratio=teacher_forcing_ratio,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    device=device,
    threshold=threshold
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.113968   | 96.854696  | 0.104082 | 95.96  | 944.92
   2    |   0.098690   | 97.147642  | 0.096169 | 95.44  | 934.91
   3    |   0.093983   | 97.255994  | 0.092946 | 95.61  | 930.42
   4    |   0.090393   | 97.338947  | 0.089466 | 95.51  | 932.07
   5    |   0.087412   | 97.396853  | 0.087315 | 95.25  | 932.76
   6    |   0.084377   | 97.442743  | 0.083859 | 95.39  | 933.25
   7    |   0.083090   | 97.422039  | 0.082618 | 95.30  | 932.71
   8    |   0.079110   | 97.484760  | 0.078668 | 95.41  | 937.60
   9    |   0.076830   | 97.528040  | 0.077242 | 95.30  | 939.33
  10    |   0.074671   | 97.566198  | 0.074740 | 95.28  | 935.49
  11    |   0.073825   | 97.587613  | 0.079615 | 95.08  | 933.01
  12    |   0.077091   | 97.561797  | 0.078089 | 95.18  | 935.99
  13    |   0.075065   | 97.609658  | 0.076396 | 95.2

In [15]:
mu.saveModel(overall_title, title, model)

In [16]:
model = mu.getModel(overall_title, title)
print(model)

MySeq2Seq(
  (prefixEncoder): Encoder(
    (embedding): Embedding(214, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (postfixEncoder): Encoder(
    (embedding): Embedding(214, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
    (hidden_fc): Linear(in_features=200, out_features=100, bias=True)
    (cell_fc): Linear(in_features=200, out_features=100, bias=True)
    (dp): Dropout(p=0.0, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(214, 100)
    (lstm): LSTM(100, 200, batch_first=True, bidirectional=True)
  )
  (pointer): PointerNetwork(
    (fc1): Linear(in_features=800, out_features=400, bias=True)
    (fc2): Linear(in_features=400, out_features=200, bias=True)
    (fc3): Linear(in_features=200, out_features=1, bias=True)
    (dp1): Dropou

In [17]:
loss, acc, TT_acc = tester.test(
    test_dataloader=test_dataloader,
    model=model,
    loss_fn=loss_fn,
    device=device,
    fn=overall_title,
    proj_nm=title,
    threshold=threshold
)

test loss:  0.06672941134099813
test acc:  97.87936731557376
TT acc:  48.31967213114754
saved precision and recall results to file!


In [18]:
with open('../stat/'+overall_title, 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\t |\t TT acc: ' + str(TT_acc)
        # text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(0.0, 3)) + ' min\t |\t TT acc: ' + str(TT_acc)
        f.write(text)

In [19]:
mu.graphModel(train_dataloader, model, writer, device)

uploaded model graph to tensorboard!
